In [6]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import networkx.algorithms.community as nx_comm
import itertools

In [7]:
concat_voted_df = list()
for date in [2019, 2020, 2021]:
    votes_df = pd.read_csv(f"datasets/votacoesVotos-{date}.csv", delimiter=";")
    votes_df = votes_df[['idVotacao',
                         'deputado_id', 'deputado_nome',
                         'deputado_siglaPartido']]
    votes_df = votes_df.rename(columns={"deputado_siglaPartido": "siglaBancada"})
    concat_voted_df.append(votes_df)

votes_gr_df = pd.concat(concat_voted_df, ignore_index=True)
votes_grouped_df = votes_gr_df.groupby(by=["deputado_id", "deputado_nome", "siglaBancada"]).agg({'idVotacao': lambda x: ','.join(x),}).reset_index()
dep_sigla_dict = dict(zip(votes_grouped_df['deputado_id'], votes_grouped_df['siglaBancada']))



In [8]:
# Distance between all the nodes comparing oriented and free
f_free = open('networks/sl_liberado.json')
data_free = json.load(f_free)

f_oriented = open('networks/sl_orientado.json')
data_oriented = json.load(f_oriented)

deputy_id_diff = dict()
for node_1, approx in data_free.items():
    value_for_node = 0
    for node_2, value_l in approx.items():
        value_o = data_oriented.get(node_1, {}).get(node_2, 0)
        value = abs(value_l-value_o)
        value_for_node += value
    deputy_id_diff[node_1] = value_for_node

f_free.close()
f_oriented.close()

